In [1]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Procesamiento de la tabla de datos nutricionales como series de tiempo**

####**Descripción:**
Genera tablas de datos nutricionales con información de 1, 2, 3 o 4 tomas nutricionales previas para cada beneficiario, para usarlas en la modelación.
 
####**Requerimientos:**

-  Diccionario `datatypes_dictionaries/dtypes_tomas.json`, que mapea cada columna de la tabla de datos nutricionales al *datatype* de Pandas más adecuado.
- Tabla de datos nutricionales limpia `clean_data/tomas_nutricionales.parquet`.
 
####**Salidas:**

- Tablas de datos `historical_data/historico_profundidad_i.parquet` (con `i`= 1, 2, 3, 4) con los datos nutricionales de cada toma y las `i` tomas anteriores de cada beneficiario.

In [2]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
pd.options.mode.chained_assignment = None

### **Lectura del archivo**

In [3]:
t0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
# Lee la tabla de datos nutricionales limpia
tom = pd.read_parquet('clean_data/tomas_nutricionales.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19084096 entries, 0 to 19084095
Data columns (total 33 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float32       
 9   Peso                            float32       
 10  Talla                           float32       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

### **Preliminares**

In [4]:
# Duración promedio de un mes en días
dias_mes = 365.25 / 12
# Elimina las tomas nutricionales que no tienen edad asignada
tom = tom[tom['EdadMeses'].notnull()]
tom['EdadMeses'] = tom['EdadMeses'].round(0)
# Conservar solo las columnas cuantitativas más informativas
cols_imp = ['IdBeneficiario', 'EdadMeses', 'MedicionPerimetroBraquial', 'Peso', 'Talla', 
            'ZScoreTallaEdad', 'ZScorePesoEdad', 'ZScorePesoTalla', 'ZScoreIMC']

### **Creación de series de tiempo**

Las columnas de tomas previas se identifican con un prefijo `-n` (con `n` = 1, 2, 3, 4) que indica que pertenece a una toma nutricional de `n` periodos atrás.


In [5]:
# Se realizarán tablas de datos que incluyan datos de 1, 2, 3 o 4 tomas anteriores
for profundidad in [1, 2, 3, 4]:
  his = tom[['IdToma'] + cols_imp]
  # Para cada periodo tomado en cuenta (de 1 a 'profundidad')
  for i in range(profundidad):
    periodo = i + 1
    # Coloca al frente de cada fila, los datos de la fila que está 'periodo' filas antes
    for col in cols_imp:
      his[f'{col}-{periodo}'] = his[col].shift(periodo)
  # Conserva solo las filas donde todos los datos pertenecen al mismo beneficiario
  his = his.loc[his['IdBeneficiario'] == his[f'IdBeneficiario-{profundidad}']]
  # Calcula el tiempo desde la última toma redondeado al número de meses
  his['tiempo'] = (his['EdadMeses'] - his['EdadMeses-1']).round(0)
  # Añade las columnas 'desnutricion_previa' (predictor) y 'desnutricion' (objetivo)
  his = his.merge(tom[['IdToma', 'desnutricion_previa', 'desnutricion']], on='IdToma', how='left')
  # Elimina las columnas de la toma actual que no se quieren predecir
  cols_drop = [col for col in cols_imp if col not in ['IdBeneficiario', 'EdadMeses']]
  his = his.drop(columns=cols_drop)
  # Guarda la tabla de datos
  his.to_parquet(f'historical_data/historico_profundidad_{profundidad}.parquet')
  print(f'Dataset histórico con profundidad de {profundidad} toma(s): {time.time() - t0:.2f} s.')

Dataset histórico con profundidad de 1 toma(s): 49.61 s.
Dataset histórico con profundidad de 2 toma(s): 82.53 s.
Dataset histórico con profundidad de 3 toma(s): 122.31 s.
Dataset histórico con profundidad de 4 toma(s): 161.09 s.
